# 101: Swissmetro MNL Mode Choice

In [ ]:
# TEST
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=12)
from pytest import approx

import larch as lx

In [ ]:
import larch as lx

This example is a mode choice model built using the Swissmetro example dataset.
First we can create a Model object:

In [ ]:
m = lx.Model()

We can attach a title to the model. The title does not affect the calculations
as all; it is merely used in various output report styles.

In [ ]:
m.title = "swissmetro example 01 (simple logit)"

We need to identify the availability and choice variables.
The Swissmetro dataset, as with all Biogeme data, is only
in `co` format, so we must define alternative
availability as an expression for each alternative, using a
dictionary to map alternative codes and expressions.

In [ ]:
m.availability_co_vars = {
    1: "TRAIN_AV * (SP!=0)",
    2: "SM_AV",
    3: "CAR_AV * (SP!=0)",
}

In the Swissmetro example dataset, as in many discrete choice
modeling applications, there is one and only one chosen
alternative for each case, so the choices can be described
as a single expression that evaluates to the code of the
chosen alternative.

In [ ]:
m.choice_co_code = "CHOICE"

We will also write utility functions for each alternative.
Since the data is only in `co` format, we must use only the
`utility_co` form for the utility functions.

In [ ]:
from larch import P, X

m.utility_co[1] = P("ASC_TRAIN")
m.utility_co[2] = 0
m.utility_co[3] = P("ASC_CAR")
m.utility_co[1] += X("TRAIN_TT") * P("B_TIME")
m.utility_co[2] += X("SM_TT") * P("B_TIME")
m.utility_co[3] += X("CAR_TT") * P("B_TIME")
m.utility_co[1] += X("TRAIN_CO*(GA==0)") * P("B_COST")
m.utility_co[2] += X("SM_CO*(GA==0)") * P("B_COST")
m.utility_co[3] += X("CAR_CO") * P("B_COST")

Larch will find all the parameters in the model, but we'd like to output them in
a rational order.  We can use the ordering method to do this:

In [ ]:
m.ordering = [
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "LOS",
        "B_.*",
    ),
]

Now we can prepare the data, which is available in the data warehouse that
comes with Larch.

In [ ]:
import pandas as pd

raw_data = pd.read_csv(lx.example_file("swissmetro.csv.gz")).rename_axis(index="CASEID")
raw_data.head()

The swissmetro example models exclude some observations.  We can use pandas
to identify the observations we would like to keep.

In [ ]:
keep = raw_data.eval("PURPOSE in (1,3) and CHOICE != 0")
selected_data = raw_data[keep]

When you've created the data you need, you can pass the dataframe to
the larch.DataFrames constructor.  Since the *swissmetro* data is in
``idco`` format, we'll need to explicitly identify the alternative
codes as well.

In [ ]:
ds = lx.Dataset.construct.from_idco(selected_data, alts={1: "Train", 2: "SM", 3: "Car"})

In [ ]:
ds

You might notice we have not carefully constructed this object to
include only the relevant data or the various simple transformations
used in the utility definition above.  Larch can do this itself, if
you assign this DataFrames not as the actual set of data used in model
estimation, but rather as the `dataservice` that can be used as the
source to create these computational arrays.

In [ ]:
m.datatree = ds

We can estimate the models and check the results match up with those given by Biogeme:

In [ ]:
m.set_cap(15)
result = m.maximize_loglike(method="SLSQP")

In [ ]:
# TEST
assert result.loglike == approx(-5331.252006971916)

In [ ]:
m.calculate_parameter_covariance();

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
assert m.pvals == approx([-0.15475, -0.70178, -0.010842, -0.012776], rel=1e-3)
assert m.pstderr == approx([0.0432357, 0.0548783, 0.000518335, 0.000568829], rel=1e-3)
assert m.parameter_summary().data["t Stat"].values.astype(float) == approx(
    [-3.58, -12.79, -20.92, -22.46], rel=1e-2
)